In [2]:
import numpy as np
from scanimagetiffio import SITiffIO
#import utils_image from the parent folder
import sys
sys.path.append("..")
from utils_image import UnrotateCropFrame, RegFrame

In [ ]:
#load the zstack tiff and calculate the mean frames for each stack

zstacktiff = '/home/zilong/Desktop/2D2P/Data/162_10072023/162_zstack__00001.tif'
ztstackrelog = '/home/zilong/Desktop/2D2P/Data/162_10072023/REdata_20230706_164845.txt'

S = SITiffIO()
S.open_tiff_file(zstacktiff, "r")
S.open_rotary_file(ztstackrelog)
S.interp_times()  # might take a while...

#get all frames and angles in S
Array = []; 
for i in range(S.get_n_frames()):
    Array.append(S.get_frame(i+1))
#change Array to a numpy array
Array = np.array(Array)
Angle = S.get_all_theta()

# unrotate each frame in the tiff file with the detected rotation center
unrotcropFrames = UnrotateCropFrame(Array, Angle, rotCenter=[256,256])


num_v=5; num_s=41; num_f=10
# reshape the unrotFrames to a 5D array
unrotcropFrames = unrotcropFrames.reshape(
    num_v, num_s, num_f, unrotcropFrames.shape[1], unrotcropFrames.shape[2]
)
# average across the first and third dimension
meanZstacks = np.mean(unrotcropFrames, axis=(0, 2))
             

In [ ]:
#load the imaging tiff 
imagingtiff = '/home/zilong/Desktop/2D2P/Data/162/162_00001.tif'
imagingrelog = '/home/zilong/Desktop/2D2P/Data/162/REdata_20230706_164845.txt'

S = SITiffIO()
S.open_tiff_file(imagingtiff, "r")
S.open_rotary_file(imagingrelog)
#extract the last self.numFrames frames from the tiff file
tailArray, tailAng = S.tail(1000)
        
# unrotate each frame in the tiff file with the detected rotation center
unrotFrames  = UnrotateCropFrame(tailArray, tailAng, rotCenter=[256,256])

#perform image registraion
meanRegImg, regFrames = RegFrame(unrotFrames)